In [25]:
import pandas as pd
da = pd.read_csv('/content/drive/MyDrive/Faculdade./TCC./Silhouette./Data/l_5_20_5000.dat')

In [26]:
nome_arquivo = 'l_5_20_5000.dat'

# Salvando o DataFrame em um arquivo de texto
da.to_csv(nome_arquivo, sep=',', index=False)

In [20]:
# Ler o conteúdo original do arquivo
with open('l_5_20_5000.dat', 'r') as arquivo:
    conteudo_original = arquivo.read()

# Abrir o arquivo no modo de escrita
with open('l_5_20_5000.dat', 'w') as arquivo:
    # Escrever a nova linha no começo do arquivo
    nova_linha = '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,t\n'
    arquivo.write(nova_linha)

    # Escrever o conteúdo original após a nova linha
    arquivo.write(conteudo_original)

# Arquivo atualizado com a nova linha no começo


In [27]:
%%writefile silhouette.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <cuda.h>



/*iris
#define DATASET "digits_ord.data"
#define NUM_CLUSTERS 10
#define NUM_FEATURES 16
#define MAX_INSTANCES 10991


#define DATASET "electricity_ord.data"
#define NUM_CLUSTERS 2
#define NUM_FEATURES 8
#define MAX_INSTANCES 45311

#define DATASET "iris.data"
#define NUM_CLUSTERS 3
#define NUM_FEATURES 4
#define MAX_INSTANCES 150

#define DATASET "poker-hand_ord.data"
#define NUM_CLUSTERS 10
#define NUM_FEATURES 10
#define MAX_INSTANCES 1025010

#define DATASET "dataset1m.csv"
#define NUM_CLUSTERS 10
#define NUM_FEATURES 16
#define MAX_INSTANCES 1000000
*/
#define DATASET "l_5_20_5000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 5000
/*
#define DATASET "l_5_20_10000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 10000

#define DATASET "l_5_20_25000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 250000

#define DATASET "l_5_20_50000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 50000

#define DATASET "l_5_20_500000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 5000000

#define DATASET "l_5_20_2500000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 2500000

#define DATASET "l_5_20_5000000.dat"
#define NUM_CLUSTERS 5
#define NUM_FEATURES 20
#define MAX_INSTANCES 5000000
*/
//512 256
#define MAX_BLOCKS 40
#define BLOCK_SIZE 128

#define CUDA_ERROR(ans) { cudaAssert((ans), __FILE__, __LINE__); }
inline void cudaAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"CUDA Error: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

typedef struct {
    double sum[NUM_FEATURES];
    int count;
} Centroid;

typedef struct {
    double features[NUM_FEATURES];
    int cluster;
} Instance;

int read_dataset(const char *filename, Instance *instances) {
    FILE *file = fopen(filename, "r");
    if (file == NULL) {
        printf("Não foi possível abrir o arquivo %s\n", filename);
        return -1;
    }

    int i = 0;
    while (!feof(file) && i < MAX_INSTANCES) {
        for (int j = 0; j < NUM_FEATURES; j++) {
            if (fscanf(file, "%lf,", &instances[i].features[j]) != 1) {
                printf("Erro ao ler a característica %d da instância %d\n", j+1, i+1);
                fclose(file);
                return -1;
            }
        }
        if (fscanf(file, "%d\n", &instances[i].cluster) != 1) {
            printf("Erro ao ler o cluster da instância %d\n", i+1);
            fclose(file);
            return -1;
        }
        i++;
    }

    fclose(file);
    return i; // Retorna o número de instâncias lidas
}

int* find_cluster_starts(Instance *data, int n_points, int n_clusters) {
    // Alocar memória para o array de índices de início do cluster
    int* cluster_starts = (int*)malloc(n_clusters * sizeof(int));
    if (cluster_starts == NULL) {
        return NULL;
    }

    // Inicializar o primeiro cluster
    int current_cluster = data[0].cluster;
    cluster_starts[current_cluster] = 0;

    // Loop através dos pontos
    for (int i = 1; i < n_points; i++) {
        // Se o cluster do ponto atual é diferente do cluster anterior, armazene o índice
        if (data[i].cluster != current_cluster) {
            current_cluster = data[i].cluster;
            cluster_starts[current_cluster] = i+1;
        }
    }

    return cluster_starts;
}

__device__ double atomicAddDouble(double* address, double val)
{
    unsigned long long int* address_as_ull =
                              (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;
    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                        __double_as_longlong(val +
                               __longlong_as_double(assumed)));
    } while (assumed != old);
    return __longlong_as_double(old);
}

__global__ void calculate_cluster_centroid(int start, int end, Instance *instances, double *centroid, int num_features) {
    extern __shared__ double sdata[];

    // each thread loads one element from global to shared memory
    unsigned int tid = threadIdx.x;
    unsigned int i = start + blockIdx.x * blockDim.x + threadIdx.x;

    for (int f = 0; f < num_features; f++) {
        sdata[tid * num_features + f] = (i < end) ? instances[i].features[f] : 0;
    }

    __syncthreads();

    // do reduction in shared memory
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            for (int f = 0; f < num_features; f++) {
                sdata[tid * num_features + f] += sdata[(tid + s) * num_features + f];
            }
        }
        __syncthreads();
    }

    // write result for this block to global memory
    if (tid == 0) {
        for (int f = 0; f < num_features; f++) {
            atomicAddDouble(&centroid[f], sdata[f]);
        }
    }
}

double euclidean_distance(double* a, double* b, int num_features) {
    double sum = 0.0;
    for (int i = 0; i < num_features; i++) {
        double diff = a[i] - b[i];
        sum += diff * diff;
    }
    return sqrt(sum);
}

double interMean(double* centroids, double* general_centroid, int num_clusters, int num_features) {
    double sum = 0.0;
    for (int c = 0; c < num_clusters; c++) {
        sum += euclidean_distance(centroids + c * num_features, general_centroid, num_features);
    }
    return sum / num_clusters;
}

__device__ double euclidean_distanceg(double* a, double* b, int num_features) {
    double sum = 0.0;
    for (int i = 0; i < num_features; i++) {
        double diff = a[i] - b[i];
        sum += diff * diff;
    }
    return sqrt(sum);
}

__global__ void calculate_intra_cluster_distance(int start, int end, Instance *instances, double *centroid, int num_features, double *result) {
    extern __shared__ double sdata[];

    // each thread loads one element from global to shared memory
    unsigned int tid = threadIdx.x;
    unsigned int i = start + blockIdx.x * blockDim.x + threadIdx.x;

    sdata[tid] = (i < end) ? euclidean_distanceg(instances[i].features, centroid, num_features) : 0;
    __syncthreads();

    // do reduction in shared memory
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    // write result for this block to global memory
    if (tid == 0) atomicAddDouble(result, sdata[0]);
}


int main() {
    clock_t start, end;
    double cpu_time_used;

    Instance *instances = (Instance*)malloc(sizeof(Instance) * MAX_INSTANCES);



    //leitura do dataset
    int num_instances = read_dataset(DATASET, instances);
    if (num_instances < 0) {
        printf("Erro ao ler o dataset\n");
        return -1;
    }
    /*
    for(int i = 0; i < MAX_INSTANCES; i++) {
        printf("Instance %d, Cluster %d, Features: ", i+1, instances[i].cluster);
        for(int j = 0; j < NUM_FEATURES; j++) {
            printf("%f ", instances[i].features[j]);
        }
        printf("\n");
    }
*/


    double *centroids = (double*)malloc(sizeof(double) * NUM_FEATURES * NUM_CLUSTERS);
    Instance *d_instances;
    double *d_centroids;
    cudaMalloc(&d_instances, sizeof(Instance) * MAX_INSTANCES);
    cudaMemcpy(d_instances, instances, sizeof(Instance) * MAX_INSTANCES, cudaMemcpyHostToDevice);
    cudaMalloc(&d_centroids, sizeof(double) * NUM_FEATURES * NUM_CLUSTERS);
    double *intra_cluster_distances = (double*)malloc(sizeof(double) * NUM_CLUSTERS);
    double *d_intra_cluster_distances;
    cudaMalloc(&d_intra_cluster_distances, sizeof(double) * NUM_CLUSTERS);


    int* cluster_starts = find_cluster_starts(instances, MAX_INSTANCES, NUM_CLUSTERS);
    if (cluster_starts == NULL) {
        printf("Failed to find cluster starts.\n");
        return -1;
    }
    start = clock();
    for (int c = 0; c < NUM_CLUSTERS; c++) {
        int start = cluster_starts[c];
        int end = (c == NUM_CLUSTERS - 1) ? MAX_INSTANCES : cluster_starts[c + 1];
        calculate_cluster_centroid<<<(end - start + BLOCK_SIZE - 1) / BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE * NUM_FEATURES * sizeof(double)>>>(start, end, d_instances, d_centroids + c * NUM_FEATURES, NUM_FEATURES);

    }
    cudaDeviceSynchronize();
    cudaMemcpy(centroids, d_centroids, sizeof(double) * NUM_FEATURES * NUM_CLUSTERS, cudaMemcpyDeviceToHost);
/*
    for (int c = 0; c < NUM_CLUSTERS; c++) {
        printf("Centroid %d: ", c+1);
        for(int f = 0; f < NUM_FEATURES; f++) {
            printf("%f ", centroids[c * NUM_FEATURES + f]);
        }
        printf("\n");
    }
*/
    for (int c = 0; c < NUM_CLUSTERS; c++) {
        int start = cluster_starts[c];
        int end = (c == NUM_CLUSTERS - 1) ? MAX_INSTANCES : cluster_starts[c + 1];
        int cluster_size = end - start;
        printf("Centroid %d: ", c+1);
        for(int f = 0; f < NUM_FEATURES; f++) {
            centroids[c * NUM_FEATURES + f] /= cluster_size;
            printf("%f ", centroids[c * NUM_FEATURES + f]);
        }
        printf("\n");
    }


    double general_centroid[NUM_FEATURES] = {0.0};

    // Cálculo do centroide geral após a obtenção dos centroides
    for (int f = 0; f < NUM_FEATURES; f++) {
        for (int c = 0; c < NUM_CLUSTERS; c++) {
            general_centroid[f] += centroids[c * NUM_FEATURES + f];
        }
        general_centroid[f] /= NUM_CLUSTERS;
    }

    // Imprimir o centroide geral
    printf("General centroid: ");
    for (int f = 0; f < NUM_FEATURES; f++) {
        printf("%f ", general_centroid[f]);
    }
    printf("\n");

    double inter_mean = interMean(centroids, general_centroid, NUM_CLUSTERS, NUM_FEATURES);
    printf("Inter-cluster mean distance: %f\n", inter_mean);

    cudaMemcpy(d_centroids, centroids, sizeof(double) * NUM_FEATURES * NUM_CLUSTERS, cudaMemcpyHostToDevice);

    for (int c = 0; c < NUM_CLUSTERS; c++) {
      int start = cluster_starts[c];
      int end = (c == NUM_CLUSTERS - 1) ? MAX_INSTANCES : cluster_starts[c + 1];
      calculate_intra_cluster_distance<<<(end - start + BLOCK_SIZE - 1) / BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE * sizeof(double)>>>(start, end, d_instances, d_centroids + c * NUM_FEATURES, NUM_FEATURES, d_intra_cluster_distances + c);

    }
    cudaDeviceSynchronize();
    cudaMemcpy(intra_cluster_distances, d_intra_cluster_distances, sizeof(double) * NUM_CLUSTERS, cudaMemcpyDeviceToHost);

    double intra_mean = 0.0;
    for (int c = 0; c < NUM_CLUSTERS; c++) {
        intra_mean += intra_cluster_distances[c];
    }

    intra_mean /= num_instances;

    printf("Intra-cluster mean distance: %f\n", intra_mean);

    double max_distance = (inter_mean > intra_mean) ? inter_mean : intra_mean;
    double bd_silhouette = (inter_mean - intra_mean) / max_distance;
    printf("BD-Silhouette: %f\n", bd_silhouette);

    end = clock();
    cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %lf segundos\n", cpu_time_used);

    cudaFree(d_intra_cluster_distances);
    free(intra_cluster_distances);
    cudaFree(d_instances);
    cudaFree(d_centroids);
    free(centroids);

    return 0;
}

Overwriting silhouette.cu


In [28]:
!time nvcc -o silhouette silhouette.cu -run

Centroid 1: 0.249954 0.252620 0.747678 0.750873 0.248588 0.249385 0.748239 0.250823 0.251242 0.751107 0.745676 0.251000 0.751633 0.250692 0.249984 0.248686 0.250581 0.748750 0.252327 0.749486 
Centroid 2: 0.248168 0.750342 0.748650 0.246941 0.251832 0.750940 0.750291 0.749282 0.246960 0.748006 0.750061 0.250511 0.748723 0.248661 0.752662 0.250423 0.748032 0.749818 0.748841 0.251601 
Centroid 3: 0.748165 0.252264 0.750312 0.749955 0.749346 0.751463 0.251001 0.251362 0.249427 0.252166 0.747540 0.747476 0.750393 0.248048 0.251059 0.253936 0.251330 0.248951 0.747090 0.248277 
Centroid 4: 0.750929 0.750677 0.751515 0.250371 0.748194 0.748618 0.248890 0.749942 0.749409 0.248954 0.749743 0.250905 0.748524 0.246643 0.749333 0.249433 0.249856 0.250868 0.249162 0.249858 
Centroid 5: 0.751104 0.747791 0.750044 0.748266 0.245554 0.748183 0.252834 0.250203 0.749241 0.751233 0.749031 0.249570 0.750736 0.748808 0.749889 0.250175 0.248039 0.753389 0.749701 0.251781 
General centroid: 0.549664 0.550739